In [2]:
import gradio as gr
import mediapipe as mp
import numpy as np
import cv2
import random
import pickle

mp_selfie = mp.solutions.selfie_segmentation

with open("smnist.h5", "rb") as f:
    model = pickle.load(f)

questions = [
    "What is the sign for 'Hello'?",
    "Show the sign for 'Thank You'",
    "Can you sign 'Happy'?",
    "What is the sign for 'Sorry'?",
    "Please sign 'Goodbye'"
]

def segment(image):
    image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)

    with mp_selfie.SelfieSegmentation(model_selection=0) as model:
        res = model.process(image)
        mask = np.stack((res.segmentation_mask,) * 3, axis=-1) > 0.5
        segmented_image = np.where(mask, image, cv2.blur(image, (40, 40)))

        text_to_display = random.choice(['Correct', 'Incorrect'])

        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 1
        color = (0, 255, 0) if text_to_display == 'Correct' else (0, 0, 255)
        thickness = 2
        textSize = cv2.getTextSize(text_to_display, font, font_scale, thickness)[0]

        textX = (image.shape[1] - textSize[0]) // 2
        textY = (image.shape[0] + textSize[1]) // 2


        cv2.putText(segmented_image, text_to_display, (textX, textY), font, font_scale, color, thickness, cv2.LINE_AA)

        return cv2.cvtColor(segmented_image, cv2.COLOR_BGR2RGB)

def update_question():
    return random.choice(questions)


webcam = gr.inputs.Image(shape=(640, 480), source="webcam")
output_image = gr.outputs.Image(type="pil")
question_text = gr.outputs.Textbox(label="Question")


options = ['Placeholder A', 'Placeholder B', 'Placeholder C']


with gr.Blocks(theme=gr.themes.Soft()) as demo:
    with gr.Tabs() as tabs:
        with gr.TabItem("Learning Mode", id=0):
            text_options = gr.Dropdown(options, label="Signs to learn")
            with gr.Row():
                t = gr.Textbox("The symbol for Happy Birthday is: ")
                img = gr.Image()
        with gr.TabItem("Game Mode", id=1):
            with gr.Row():
                question_display = gr.Textbox(label="Current Question", value=update_question(), interactive=False)
                next_question_button = gr.Button("Next Question")
            with gr.Row():
                webcam_input = gr.Image(source="webcam", tool="select", type="pil")
                i = gr.Interface(fn=segment, inputs=webcam_input, outputs=output_image, live=True)
            next_question_button.click(fn=update_question, inputs=[], outputs=question_display)
            

demo.launch()

/var/folders/q2/b9j3mww10tg9l3lq7zbhnn0h0000gn/T/ipykernel_42264/340765079.py:46: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  webcam = gr.inputs.Image(shape=(640, 480), source="webcam")
/var/folders/q2/b9j3mww10tg9l3lq7zbhnn0h0000gn/T/ipykernel_42264/340765079.py:46: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  webcam = gr.inputs.Image(shape=(640, 480), source="webcam")
/var/folders/q2/b9j3mww10tg9l3lq7zbhnn0h0000gn/T/ipykernel_42264/340765079.py:47: GradioDeprecationWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  output_image = gr.outputs.Image(type="pil")
/var/folders/q2/b9j3mww10tg9l3lq7zbhnn0h0000gn/T/ipykernel_42264/340765079.py:48: GradioDeprecationWarning: Usage of gradio.outputs is deprecated, and will not be supported in the futur

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
